In [1]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Set-up to have matplotlib use its support for notebook inline plots
%matplotlib inline

In [3]:
btc_dfd = pd.read_csv('Binance_BTCUSDT_d.csv')
btc_dfd

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount
0,1.669766e+12,2022-11-30 00:00:00,BTC/USDT,16442.91,16466.66,16428.30,16443.60,2323.64958,3.822050e+07,48232.0
1,1.669680e+12,2022-11-29 00:00:00,BTC/USDT,16212.18,16548.71,16100.00,16442.53,248106.25009,4.072651e+09,5206638.0
2,1.669594e+12,2022-11-28 00:00:00,BTC/USDT,16428.77,16487.04,15995.27,16212.91,252695.40367,4.096399e+09,5570407.0
3,1.669507e+12,2022-11-27 00:00:00,BTC/USDT,16457.61,16600.00,16401.00,16428.78,162025.47607,2.679049e+09,3362221.0
4,1.669421e+12,2022-11-26 00:00:00,BTC/USDT,16521.35,16701.99,16385.00,16458.57,181804.81666,3.011990e+09,3817130.0
...,...,...,...,...,...,...,...,...,...,...
1928,1.503274e+09,2017-08-21,BTC/USDT,4086.29,4119.62,3911.79,4016.00,685.12000,2.770592e+06,NaN
1929,1.503187e+09,2017-08-20,BTC/USDT,4139.98,4211.08,4032.62,4086.29,463.54000,1.915636e+06,NaN
1930,1.503101e+09,2017-08-19,BTC/USDT,4108.37,4184.69,3850.00,4139.98,371.15000,1.508239e+06,NaN
1931,1.503014e+09,2017-08-18,BTC/USDT,4285.08,4371.52,3938.77,4108.37,1178.07000,4.994494e+06,NaN


In [4]:
btc_dfd['date'] = pd.to_datetime(btc_dfd['date']).dt.date
btc_clean = btc_dfd.drop(['unix', 'symbol', 'Volume BTC', 'tradecount'], axis=1)
res_btc = btc_clean[~(btc_clean['date'] < pd.to_datetime("2018-01-01").date())]
res_btc

,date,open,high,low,close,Volume USDT
0,2022-11-30,16442.91,16466.66,16428.30,16443.60,3.822050e+07
1,2022-11-29,16212.18,16548.71,16100.00,16442.53,4.072651e+09
2,2022-11-28,16428.77,16487.04,15995.27,16212.91,4.096399e+09
3,2022-11-27,16457.61,16600.00,16401.00,16428.78,2.679049e+09
4,2022-11-26,16521.35,16701.99,16385.00,16458.57,3.011990e+09
...,...,...,...,...,...,...
1790,2018-01-05,15059.56,17176.24,14600.00,16960.39,3.693220e+08
1791,2018-01-04,14919.51,15280.00,13918.04,15059.54,3.127816e+08
1792,2018-01-03,14690.00,15307.56,14150.00,14919.51,2.361169e+08
1793,2018-01-02,13382.16,15473.49,12890.02,14675.11,2.797171e+08


In [5]:
btc = btc_clean.iloc[::-1]
btc

,date,open,high,low,close,Volume USDT
1932,2017-08-17,4469.93,4485.39,4200.74,4285.08,2.812379e+06
1931,2017-08-18,4285.08,4371.52,3938.77,4108.37,4.994494e+06
1930,2017-08-19,4108.37,4184.69,3850.00,4139.98,1.508239e+06
1929,2017-08-20,4139.98,4211.08,4032.62,4086.29,1.915636e+06
1928,2017-08-21,4086.29,4119.62,3911.79,4016.00,2.770592e+06
...,...,...,...,...,...,...
4,2022-11-26,16521.35,16701.99,16385.00,16458.57,3.011990e+09
3,2022-11-27,16457.61,16600.00,16401.00,16428.78,2.679049e+09
2,2022-11-28,16428.77,16487.04,15995.27,16212.91,4.096399e+09
1,2022-11-29,16212.18,16548.71,16100.00,16442.53,4.072651e+09


In [6]:
btc = btc[['open', 'high', 'low', 'Volume USDT', 'close']]

In [7]:
model_data = btc.reset_index(drop=True)
model_data

,open,high,low,Volume USDT,close
0,4469.93,4485.39,4200.74,2.812379e+06,4285.08
1,4285.08,4371.52,3938.77,4.994494e+06,4108.37
2,4108.37,4184.69,3850.00,1.508239e+06,4139.98
3,4139.98,4211.08,4032.62,1.915636e+06,4086.29
4,4086.29,4119.62,3911.79,2.770592e+06,4016.00
...,...,...,...,...,...
1928,16521.35,16701.99,16385.00,3.011990e+09,16458.57
1929,16457.61,16600.00,16401.00,2.679049e+09,16428.78
1930,16428.77,16487.04,15995.27,4.096399e+09,16212.91
1931,16212.18,16548.71,16100.00,4.072651e+09,16442.53


In [15]:
btc.duplicated().sum()

0

In [16]:
duplicates = btc[btc.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [open, high, low, Volume USDT, close]
Index: []


In [17]:
# No duplicated data

In [18]:
btc.isnull().sum()

open           0
high           0
low            0
Volume USDT    0
close          0
dtype: int64

In [19]:
# No missing value

In [8]:
scaler = MinMaxScaler()
close_df = scaler.fit_transform(np.array(model_data))

In [9]:
close_df

array([[1.99094453e-02, 1.83935731e-02, 2.18236933e-02, 1.24996525e-04,
        1.70362814e-02],
       [1.70362841e-02, 1.66610117e-02, 1.76920262e-02, 2.72862138e-04,
        1.42896423e-02],
       [1.42896445e-02, 1.38183450e-02, 1.62919877e-02, 3.66247078e-05,
        1.47809629e-02],
       ...,
       [2.05788134e-01, 2.01001773e-01, 2.07841446e-01, 2.77516787e-01,
        2.02432946e-01],
       [2.02421631e-01, 2.01940098e-01, 2.09493198e-01, 2.75907595e-01,
        2.06001976e-01],
       [2.06007915e-01, 2.00691686e-01, 2.14670990e-01, 2.52434099e-03,
        2.06018607e-01]])

In [10]:
training_size = int(len(close_df)*0.9)
test_size = len(close_df)-training_size
train_data,test_data = close_df[0:training_size,:],close_df[training_size:len(close_df),:]
print('train_data: ', train_data.shape)
print('test_data: ', test_data.shape)

train_data:  (1739, 5)
test_data:  (194, 5)


In [11]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX = []
    dataY = []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step)]  
        b = dataset[i+time_step][4]
        # print(a)
        # print(type(b))
        dataX.append(a)
        dataY.append(float(b))
    
    return np.array(dataX), np.array(dataY)

In [12]:
time_step = 10
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)
# reshape input to be [samples, time steps, features] which is required for LSTM
# X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
# X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (1728, 10, 5)
X_test:  (183, 10, 5)


In [13]:
X_train

array([[[1.99094453e-02, 1.83935731e-02, 2.18236933e-02, 1.24996525e-04,
         1.70362814e-02],
        [1.70362841e-02, 1.66610117e-02, 1.76920262e-02, 2.72862138e-04,
         1.42896423e-02],
        [1.42896445e-02, 1.38183450e-02, 1.62919877e-02, 3.66247078e-05,
         1.47809629e-02],
        ...,
        [1.43773082e-02, 1.66634461e-02, 1.99984544e-02, 1.52984262e-04,
         1.75170326e-02],
        [1.75170354e-02, 1.79145968e-02, 2.25608544e-02, 9.97851124e-05,
         1.69678913e-02],
        [1.69678940e-02, 1.65922387e-02, 2.20077470e-02, 0.00000000e+00,
         1.78501234e-02]],

       [[1.70362841e-02, 1.66610117e-02, 1.76920262e-02, 2.72862138e-04,
         1.42896423e-02],
        [1.42896445e-02, 1.38183450e-02, 1.62919877e-02, 3.66247078e-05,
         1.47809629e-02],
        [1.47809652e-02, 1.42198757e-02, 1.91721841e-02, 6.42309598e-05,
         1.39464484e-02],
        ...,
        [1.75170354e-02, 1.79145968e-02, 2.25608544e-02, 9.97851124e-05,
        

In [14]:
y_train

array([0.01742377, 0.01861563, 0.02173655, ..., 0.4236441 , 0.39676042,
       0.42169032])

In [21]:
train_X = X_train
train_y = y_train
test_X = X_test
test_y = y_test

In [22]:
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(1728, 10, 5) (1728,) (183, 10, 5) (183,)


## Model Training

In [23]:
hyperparams = []
batch = [16, 32, 64]
epoch = [25, 50, 100]
neuron = [50, 60, 100]
for j in batch:
    for k in epoch:
        for l in neuron:
            hyperparams.append((j,k,l))
hyperparams

[(16, 25, 50),
 (16, 25, 60),
 (16, 25, 100),
 (16, 50, 50),
 (16, 50, 60),
 (16, 50, 100),
 (16, 100, 50),
 (16, 100, 60),
 (16, 100, 100),
 (32, 25, 50),
 (32, 25, 60),
 (32, 25, 100),
 (32, 50, 50),
 (32, 50, 60),
 (32, 50, 100),
 (32, 100, 50),
 (32, 100, 60),
 (32, 100, 100),
 (64, 25, 50),
 (64, 25, 60),
 (64, 25, 100),
 (64, 50, 50),
 (64, 50, 60),
 (64, 50, 100),
 (64, 100, 50),
 (64, 100, 60),
 (64, 100, 100)]

In [24]:
hyperparam1 = hyperparams[:9]
hyperparam2 = hyperparams[9:18]
hyperparam3 = hyperparams[18:27]
hyperparam1

[(16, 25, 50),
 (16, 25, 60),
 (16, 25, 100),
 (16, 50, 50),
 (16, 50, 60),
 (16, 50, 100),
 (16, 100, 50),
 (16, 100, 60),
 (16, 100, 100)]

In [25]:
hyperparam2

[(32, 25, 50),
 (32, 25, 60),
 (32, 25, 100),
 (32, 50, 50),
 (32, 50, 60),
 (32, 50, 100),
 (32, 100, 50),
 (32, 100, 60),
 (32, 100, 100)]

In [26]:
hyperparam3

[(64, 25, 50),
 (64, 25, 60),
 (64, 25, 100),
 (64, 50, 50),
 (64, 50, 60),
 (64, 50, 100),
 (64, 100, 50),
 (64, 100, 60),
 (64, 100, 100)]

In [27]:
lstms1 = []
models1 = []
for batch, epoch, neuron in hyperparam1:
    model = Sequential()
    model.add(LSTM(neuron, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(units=1))
    model.compile(loss='mse',optimizer='adam')
    lstm = model.fit(train_X, train_y, epochs=epoch, batch_size=batch, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    lstms1.append(lstm)
    models1.append(model)

Metal device set to: Apple M1


2023-02-15 21:11:42.204865: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-15 21:11:42.206601: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/25


2023-02-15 21:11:42.597674: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-15 21:11:42.989288: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-15 21:11:43.069942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-15 21:11:43.958111: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-15 21:11:45.132922: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-15 21:11:45.165772: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


108/108 - 3s - loss: 0.0060 - val_loss: 0.0026 - 3s/epoch - 25ms/step
Epoch 2/25
108/108 - 1s - loss: 0.0050 - val_loss: 0.0090 - 720ms/epoch - 7ms/step
Epoch 3/25
108/108 - 1s - loss: 0.0025 - val_loss: 0.0042 - 714ms/epoch - 7ms/step
Epoch 4/25
108/108 - 1s - loss: 0.0014 - val_loss: 0.0030 - 728ms/epoch - 7ms/step
Epoch 5/25
108/108 - 1s - loss: 0.0011 - val_loss: 0.0022 - 743ms/epoch - 7ms/step
Epoch 6/25
108/108 - 1s - loss: 0.0011 - val_loss: 0.0018 - 736ms/epoch - 7ms/step
Epoch 7/25
108/108 - 1s - loss: 0.0011 - val_loss: 0.0016 - 895ms/epoch - 8ms/step
Epoch 8/25
108/108 - 1s - loss: 0.0011 - val_loss: 0.0015 - 734ms/epoch - 7ms/step
Epoch 9/25
108/108 - 1s - loss: 0.0010 - val_loss: 0.0014 - 813ms/epoch - 8ms/step
Epoch 10/25
108/108 - 1s - loss: 0.0010 - val_loss: 0.0014 - 738ms/epoch - 7ms/step
Epoch 11/25
108/108 - 1s - loss: 9.7347e-04 - val_loss: 0.0013 - 722ms/epoch - 7ms/step
Epoch 12/25
108/108 - 1s - loss: 9.2948e-04 - val_loss: 0.0013 - 742ms/epoch - 7ms/step
Epoch 

...

108/108 - 1s - loss: 0.0041 - val_loss: 0.0023 - 749ms/epoch - 7ms/step
Epoch 3/100
108/108 - 1s - loss: 0.0027 - val_loss: 0.0025 - 732ms/epoch - 7ms/step
Epoch 4/100
108/108 - 1s - loss: 0.0016 - val_loss: 0.0013 - 731ms/epoch - 7ms/step
Epoch 5/100
108/108 - 1s - loss: 0.0012 - val_loss: 8.1736e-04 - 727ms/epoch - 7ms/step
Epoch 6/100
108/108 - 1s - loss: 0.0010 - val_loss: 6.3020e-04 - 729ms/epoch - 7ms/step
Epoch 7/100
108/108 - 1s - loss: 9.5350e-04 - val_loss: 5.6602e-04 - 733ms/epoch - 7ms/step
Epoch 8/100
108/108 - 1s - loss: 9.2525e-04 - val_loss: 5.4971e-04 - 734ms/epoch - 7ms/step
Epoch 9/100
108/108 - 1s - loss: 8.9901e-04 - val_loss: 5.4564e-04 - 725ms/epoch - 7ms/step
Epoch 10/100
108/108 - 1s - loss: 8.6736e-04 - val_loss: 5.4488e-04 - 722ms/epoch - 7ms/step
Epoch 11/100
108/108 - 1s - loss: 8.3231e-04 - val_loss: 5.4559e-04 - 719ms/epoch - 7ms/step
Epoch 12/100
108/108 - 1s - loss: 7.9724e-04 - val_loss: 5.4850e-04 - 725ms/epoch - 7ms/step
Epoch 13/100
108/108 - 1s - l

2023-02-15 21:17:04.107834: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-15 21:17:04.181115: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-15 21:17:04.257084: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


108/108 - 2s - loss: 0.0014 - val_loss: 0.0079 - 2s/epoch - 14ms/step
Epoch 2/100


2023-02-15 21:17:05.193355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-15 21:17:05.222028: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


108/108 - 1s - loss: 0.0031 - val_loss: 9.2631e-04 - 741ms/epoch - 7ms/step
Epoch 3/100
108/108 - 1s - loss: 0.0038 - val_loss: 0.0059 - 719ms/epoch - 7ms/step
Epoch 4/100
108/108 - 1s - loss: 0.0027 - val_loss: 0.0046 - 722ms/epoch - 7ms/step
Epoch 5/100
108/108 - 1s - loss: 0.0015 - val_loss: 0.0028 - 765ms/epoch - 7ms/step
Epoch 6/100
108/108 - 1s - loss: 0.0011 - val_loss: 0.0018 - 739ms/epoch - 7ms/step
Epoch 7/100
108/108 - 1s - loss: 8.8959e-04 - val_loss: 0.0013 - 728ms/epoch - 7ms/step
Epoch 8/100
108/108 - 1s - loss: 8.2264e-04 - val_loss: 0.0011 - 734ms/epoch - 7ms/step
Epoch 9/100
108/108 - 1s - loss: 8.0071e-04 - val_loss: 0.0010 - 734ms/epoch - 7ms/step
Epoch 10/100
108/108 - 1s - loss: 7.8630e-04 - val_loss: 9.8453e-04 - 746ms/epoch - 7ms/step
Epoch 11/100
108/108 - 1s - loss: 7.6732e-04 - val_loss: 9.5039e-04 - 733ms/epoch - 7ms/step
Epoch 12/100
108/108 - 1s - loss: 7.4337e-04 - val_loss: 9.1709e-04 - 730ms/epoch - 7ms/step
Epoch 13/100
108/108 - 1s - loss: 7.1751e-04 

In [28]:
%run Evaluation.ipynb

In [37]:
i=0
for x in models1:
    # make a prediction
    # test_x2 = test_X
    # yhat = x.predict(test_x2)
    # test_x2 = test_x2.reshape((test_x2.shape[0], test_x2.shape[2]))
    # # invert scaling for forecast
    # inv_yhat = np.concatenate((test_x2[:, :],yhat), axis=1)
    # inv_yhat = scaler.inverse_transform(inv_yhat)
    # # inv_yhat = inv_yhat[:,0]
    # # invert scaling for actual
    # test_y = test_y.reshape((len(test_y), 1))
    # inv_y = np.concatenate((test_x2[:, :], test_y), axis=1)
    # inv_y = scaler.inverse_transform(inv_y)
    # # inv_y = inv_y[:,0]
    # yhat_df = pd.DataFrame(inv_yhat, columns = ['Open','High','Low', 'Volume', 'Close', 'Target'])
    # y_df = pd.DataFrame(inv_y, columns = ['Open','High','Low', 'Volume', 'Close', 'Target'])
    # print(hyperparam1[i])
    # print("Epoch: "+ str(lstms1[i].params['epochs']))
    # print("Neurons: "+str(x.layers[0].units))
    # # print(str(lstms[i].params))
    # i = i+1
    test_x2 = test_X
    test_x3 = x.predict(test_x2)
    # print(test_x3.shape)
    # print(test_y.shape)
    print('RMSE')
    print(RMSE(test_x3,test_y))
    print('MAE')
    print(MAE(test_x3,test_y))
    print('MAPE')
    print(MAPE(test_x3,test_y))

6/6 [==============================] - 0s 4ms/step
RMSE
[0.03457522]
MAE
[0.03071824]
MAPE
[10.283894]
6/6 [==============================] - 0s 4ms/step
RMSE
[0.02943179]
MAE
[0.02579465]
MAPE
[8.810367]
6/6 [==============================] - 0s 8ms/step
RMSE
[0.03060083]
MAE
[0.02708519]
MAPE
[9.187714]
6/6 [==============================] - 0s 4ms/step
RMSE
[0.03522087]
MAE
[0.03176555]
MAPE
[10.602008]
6/6 [==============================] - 0s 6ms/step
RMSE
[0.03609582]
MAE
[0.03248367]
MAPE
[10.7929325]
6/6 [==============================] - 0s 4ms/step
RMSE
[0.03399112]
MAE
[0.03023492]
MAPE
[10.155189]
6/6 [==============================] - 0s 5ms/step
RMSE
[0.03999243]
MAE
[0.03614402]
MAPE
[11.83733]
6/6 [==============================] - 0s 4ms/step
RMSE
[0.0445492]
MAE
[0.04021761]
MAPE
[12.965089]
6/6 [==============================] - 0s 4ms/step
RMSE
[0.04518748]
MAE
[0.04101871]
MAPE
[13.208814]


In [41]:
len(yhat)

183

In [51]:
i=0
for x in models1:
    # make a prediction
    test_x2 = test_X
    yhat = x.predict(test_x2)
    # test_x2 = test_x2.reshape((test_x2.shape[0], test_x2.shape[2]))
    # invert scaling for forecast
    inv_yhat = np.concatenate((np.zeros((len(yhat), 4)),yhat), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    # inv_yhat = inv_yhat[:,0]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((np.zeros((len(yhat), 4)), test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    # inv_y = inv_y[:,0]
    yhat_df = pd.DataFrame(inv_yhat, columns = ['Open','High','Low', 'Volume', 'Close'])
    y_df = pd.DataFrame(inv_y, columns = ['Open','High','Low', 'Volume', 'Close'])
    print(hyperparam1[i])
    print("Epoch: "+ str(lstms1[i].params['epochs']))
    print("Neurons: "+str(x.layers[0].units))
    # print(str(lstms[i].params))
    i = i+1
    # test_x2 = test_X
    # test_x3 = x.predict(test_x2)
    # # print(test_x3.shape)
    # # print(test_y.shape)
    # print(yhat_df)
    print('RMSE')
    print(RMSE(y_df['Close'],yhat_df['Close']))
    print('MAE')
    print(MAE(y_df['Close'],yhat_df['Close']))
    print('MAPE')
    print(MAPE(y_df['Close'],yhat_df['Close']))

6/6 [==============================] - 0s 11ms/step
(16, 25, 50)
Epoch: 25
Neurons: 50
RMSE
2224.459219284565
MAE
1976.3141181765832
MAPE
9.946139129078627
6/6 [==============================] - 0s 5ms/step
(16, 25, 60)
Epoch: 25
Neurons: 60
RMSE
1893.547573661105
MAE
1659.5458418080011
MAPE
8.358792246114923
6/6 [==============================] - 0s 4ms/step
(16, 25, 100)
Epoch: 25
Neurons: 100
RMSE
1968.7598269088612
MAE
1742.5749648187661
MAPE
8.748261509970764
6/6 [==============================] - 0s 6ms/step
(16, 50, 50)
Epoch: 50
Neurons: 50
RMSE
2265.9985149453464
MAE
2043.6941108071774
MAPE
10.25769405989276
6/6 [==============================] - 0s 4ms/step
(16, 50, 60)
Epoch: 50
Neurons: 60
RMSE
2322.290182812373
MAE
2089.8961041423513
MAPE
10.475766789899925
6/6 [==============================] - 0s 4ms/step
(16, 50, 100)
Epoch: 50
Neurons: 100
RMSE
2186.8800380868565
MAE
1945.2183353973137
MAPE
9.793090187473009
6/6 [==============================] - 0s 4ms/step
(16, 100, 

In [52]:
y_df

,Open,High,Low,Volume,Close
0,3189.02,3276.5,2817.0,967753.32,31801.04
1,3189.02,3276.5,2817.0,967753.32,29805.83
2,3189.02,3276.5,2817.0,967753.32,30452.62
3,3189.02,3276.5,2817.0,967753.32,29700.21
4,3189.02,3276.5,2817.0,967753.32,29864.04
...,...,...,...,...,...
178,3189.02,3276.5,2817.0,967753.32,16522.14
179,3189.02,3276.5,2817.0,967753.32,16458.57
180,3189.02,3276.5,2817.0,967753.32,16428.78
181,3189.02,3276.5,2817.0,967753.32,16212.91


In [55]:
model_data[1000:]

,open,high,low,Volume USDT,close
1000,8562.04,8978.26,8528.78,7.601955e+08,8810.79
1001,8810.99,9398.00,8792.99,8.397257e+08,9309.37
1002,9309.35,9939.00,9256.76,1.247457e+09,9791.98
1003,9791.97,9845.62,9150.00,1.099721e+09,9316.42
1004,9315.96,9588.00,9220.00,5.605470e+08,9381.27
...,...,...,...,...,...
1928,16521.35,16701.99,16385.00,3.011990e+09,16458.57
1929,16457.61,16600.00,16401.00,2.679049e+09,16428.78
1930,16428.77,16487.04,15995.27,4.096399e+09,16212.91
1931,16212.18,16548.71,16100.00,4.072651e+09,16442.53


In [56]:
yhat_df

,Open,High,Low,Volume,Close
0,3189.02,3276.5,2817.0,967753.32,32264.634795
1,3189.02,3276.5,2817.0,967753.32,32571.742567
2,3189.02,3276.5,2817.0,967753.32,30858.672904
3,3189.02,3276.5,2817.0,967753.32,31136.158975
4,3189.02,3276.5,2817.0,967753.32,30144.212567
...,...,...,...,...,...
178,3189.02,3276.5,2817.0,967753.32,19472.906784
179,3189.02,3276.5,2817.0,967753.32,19325.448269
180,3189.02,3276.5,2817.0,967753.32,19298.493652
181,3189.02,3276.5,2817.0,967753.32,19174.947821
